# Import + Checking Environment

In [1]:
import keras as k
import tensorflow as tf
print('tensorflow: ', tf.__version__)
print('keras: ', k.__version__)

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D


Using TensorFlow backend.


tensorflow:  2.2.0
keras:  2.3.1


In [2]:
# Limit GPU Ram
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
tf.test.is_built_with_cuda()

True

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12866530539060372514
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16494238091145842728
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6917013847798718395
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4294967296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3866630332494563000
physical_device_desc: "device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


# Preprocessing

In [3]:
# the FULL dataset

# create generator
datagen = ImageDataGenerator()

# prepare an iterators for each dataset
train_full = datagen.flow_from_directory('/home/maihai/GitHub/Fruit-Images-Dataset/train_full', 
                                          target_size=(32, 32),
                                          class_mode='categorical',
                                          shuffle=True,
                                          batch_size=8)

test_full = datagen.flow_from_directory('/home/maihai/GitHub/Fruit-Images-Dataset/test_full',
                                         target_size=(32, 32),
                                         class_mode='categorical',
                                         shuffle=True,
                                         batch_size=8)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [7]:
# the SMALLER dataset

# create generator
imgGen = ImageDataGenerator()

# prepare an iterators for each dataset
train_smaller = imgGen.flow_from_directory('train_smaller', 
                                            target_size=(32, 32),
                                            class_mode='categorical',
                                            shuffle=True,
                                            batch_size=8)
imgGen
test_smaller = datagen.flow_from_directory('test_smaller',
                                            target_size=(32, 32),
                                            class_mode='categorical',
                                            shuffle=True,
                                            batch_size=8)

Found 1409 images belonging to 3 classes.
Found 473 images belonging to 3 classes.


# The LeNet-5

In [ ]:
# LeNet-5 - output 131
lenet5 = Sequential()

lenet5.add(Conv2D(filters=6, input_shape=(32, 32, 3), kernel_size=5, strides=1, activation='relu'))
lenet5.add(AveragePooling2D())

lenet5.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
lenet5.add(AveragePooling2D())

lenet5.add(Flatten())

lenet5.add(Dense(units=120, activation='relu'))
lenet5.add(Dense(units=84, activation='relu'))

# Output
lenet5.add(Dense(units=131, activation = 'softmax'))

lenet5.summary()


In [8]:
lenet5.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
%%time
lenet5.fit(train_full, steps_per_epoch=None, epochs=100)

Epoch 1/100


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_1/convolution (defined at /home/maihai/miniconda3/envs/MLgpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1305]

Function call stack:
keras_scratch_graph


In [ ]:
print(lenet5.evaluate(test_full))

# Simple Models

In [10]:
# LeNet-5 - output 131
model = Sequential()

lenet5.add(Conv2D(filters=6, input_shape=(32, 32, 3), kernel_size=5, strides=1, activation='relu'))
lenet5.add(AveragePooling2D())

lenet5.add(Flatten())

lenet5.add(Dense(units=120, activation='relu'))
lenet5.add(Dense(units=84, activation='relu'))

# Output
lenet5.add(Dense(units=131, activation = 'softmax'))

lenet5.summary()


ValueError: Input 0 is incompatible with layer conv2d_3: expected ndim=4, found ndim=2

# Tensorboard

In [12]:
from keras.callbacks import TensorBoard
import datetime # miscellanous: to name folders

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir)

In [27]:
%%time
# LeNet5 + Tensorboard + the FULL dataset
lenet5.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=[k.metrics.CategoricalAccuracy(), # Using 2 metrics
                        k.metrics.AUC()])

lenet5.fit(train_full, 
           validation_data=test_full,
           steps_per_epoch=None, epochs=15,
           callbacks=[tensorboard_callback])

Epoch 1/15
8462/8462 [==============================] - 45s 5ms/step - loss: 0.4726 - categorical_accuracy: 0.9874 - auc_1: 0.9947 - val_loss: 24.5191 - val_categorical_accuracy: 0.8932 - val_auc_1: 0.9496
Epoch 2/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5526 - categorical_accuracy: 0.9878 - auc_1: 0.9948 - val_loss: 7.9424 - val_categorical_accuracy: 0.8999 - val_auc_1: 0.9521
Epoch 3/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5212 - categorical_accuracy: 0.9898 - auc_1: 0.9955 - val_loss: 6.8745 - val_categorical_accuracy: 0.8613 - val_auc_1: 0.9331
Epoch 4/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5245 - categorical_accuracy: 0.9902 - auc_1: 0.9956 - val_loss: 37.5403 - val_categorical_accuracy: 0.8841 - val_auc_1: 0.9434
Epoch 5/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.6070 - categorical_accuracy: 0.9904 - auc_1: 0.9956 - val_loss: 0.0000e+00 - val_categorical_accura

In [29]:
%%time
print(lenet5.evaluate(test_full))

2836/2836 [==============================] - 9s 3ms/step
[0.0, 0.8846967816352844, 0.9428174495697021]
CPU times: user 32.5 s, sys: 1.46 s, total: 33.9 s
Wall time: 9.31 s
